In [1]:
!pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 MB 18.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 59.1 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.4 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116949 sha256=fde914f2c34015f6ecd9893267a39e8e39888ad22e0b58a5bf22ba2b0aa4cb15
  Stored in directory: /root/.cache/pip/wheels/20/e8/7b/003fc14f02f262dd4614aec55e41147c8012e3dad98c936b76
Successfully built fire
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninst

In [ ]:
import pickle
import Object_Detection_ 

import tensorflow as tf
import numpy as np
import pandas as pd
import json
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM , Dense,GlobalMaxPooling1D,Flatten
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

from deepface import DeepFace

In [ ]:
#importing the dataset
with open('intents.json') as content:
    data1 = json.load(content)
#getting all the data to lists
tags = []
inputs = []
responses={}
for intent in data1['intents']:
    responses[intent['tag']]=intent['responses']
    for lines in intent['patterns']:
        inputs.append(lines)
        tags.append(intent['tag'])
#converting to dataframe
data = pd.DataFrame({"patterns":inputs,
                     "tags":tags})
data

In [ ]:
#removing punctuations
import string
data['patterns'] = data['patterns'].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])
data['patterns'] = data['patterns'].apply(lambda wrd: ''.join(wrd))
#tokenize the data
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data['patterns'])
train = tokenizer.texts_to_sequences(data['patterns'])

#apply padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train = pad_sequences(train)

#encoding the outputs
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(data['tags'])

In [ ]:
#input length
input_shape = x_train.shape[1]
print(input_shape)
#define vocabulary
vocabulary = len(tokenizer.word_index)
print("number of unique words : ",vocabulary)
#output length
output_length = le.classes_.shape[0]
print("output length: ",output_length)

In [ ]:
#creating the model
i = Input(shape=(input_shape,))
x = Embedding(vocabulary+1,300)(i)
x = LSTM(300,return_sequences=True)(x)
x = Flatten()(x)
x = Dense(output_length,activation="softmax")(x)
model  = Model(i,x)
#compiling the model
model.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
#training the model
train = model.fit(x_train,y_train,epochs=200)

In [ ]:
def getVideoClassification(VideoFeatures):
    texts_p = []
    prediction_input = VideoFeatures
    #removing punctuation and converting to lowercase
    prediction_input = [letters.lower() for letters in prediction_input if letters not in string.punctuation]
    prediction_input = ''.join(prediction_input)
    texts_p.append(prediction_input)
    
    #tokenizing and padding
    prediction_input = tokenizer.texts_to_sequences(texts_p)
    prediction_input = np.array(prediction_input).reshape(-1)
    prediction_input = pad_sequences([prediction_input],input_shape)
    
    #getting output from model
    output = model.predict(prediction_input)
    output = output.argmax()
    
    #finding the right tag and predicting
    response_tag = le.inverse_transform([output])[0]
    return responses[response_tag]

In [ ]:
def EmotionRecognition(VideoPath):
        cap = cv2.VideoCapture(videoPath)

    EmotionDetected = []

    if(cap.isOpened() == False):
        print("Error opening file ...")
    else:
        (success, image) = cap.read()

        startTime = 0

        while success:
            print("Video is running..")
            currentTime = time.time()

            fps = 1/(currentTime - startTime)
            startTime = currentTime

            result = DeepFace.analyze(image, actions=['age', 'gender', 'emotion'])
            predicted_value = result[0]['dominant_emotion']
            EmotionDetected.append(predicted_value)
            
            print(predicted_value)
            
        return EmotionDetected

In [ ]:
def main(VideoPath):
    
    # PART 1: Collecting information from object detection
    ObjectList = Object_Detection_.get_Objects(VideoPath)
    
    # PART 2: Collecting information from Emotion detection
    EmotionList = Emotion_Recognition(VideoPath)
    
    # PART 3: Video Classification using json file
    VideoFeatures = ObjectList + EmotionList
    VideoFeatures = ''.join(VideoFeatures)
    Classification_Text = getVideoClassification(VideoFeatures)
    
    # PART 4: Write description for every video
    tokens = Classification_Text[0].split(',')
    Sentiment, Topic, target, age = tokens[0], tokens[1], tokens[2], tokens[3]
    Conclusion = {}
    Conclusion["Video"] = VideoPath
    Conclusion["Sentiment"] = Sentiment
    Conclusion["Topic"] = Topic
    Conclusion["Target"] = target
    Conclusion["Age"] = age
    jsonString = json.dumps(Conclusion)
    jsonFile = open("Videosdata.json", "a")
    jsonFile.write(jsonString)
    jsonFile.close()